Break up the work in milestones

1. First have a Patients Data Table ready
    - uncover insights about the patients
    - 
    
#### Notes:
- WHERE clause goes at the end (even when selecting 2+ tables

In [ ]:
# do I have to have postgressql (the app running? )
# do I have to keep postico running?

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [6]:
import time
import glob

In [3]:
ip_address = 'mimic-db-inst.cuk5g0jpkhib.us-west-2.rds.amazonaws.com:5432'

In [4]:
cnx = create_engine('postgresql://mimic:thisismetis@'+
                     ip_address +'/MIMIC_ICU')

In [5]:
pd.read_sql_query('''SELECT * from PATIENTS limit 5''',cnx)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,612,646,M,2128-01-05,None,None,None,0
1,613,647,M,2106-03-24,None,None,None,0
2,614,648,M,2139-07-13,None,None,None,0
3,615,649,M,2177-06-23,None,None,None,0
4,616,650,M,2051-04-15,2111-12-28 00:00:00,None,2111-12-28 00:00:00,1


In [6]:
pd.read_sql_query(""" SELECT * from PATIENTS limit 5""", cnx)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,612,646,M,2128-01-05,None,None,None,0
1,613,647,M,2106-03-24,None,None,None,0
2,614,648,M,2139-07-13,None,None,None,0
3,615,649,M,2177-06-23,None,None,None,0
4,616,650,M,2051-04-15,2111-12-28 00:00:00,None,2111-12-28 00:00:00,1


In [ ]:
#'postgresql://dbadmin:thisismimic@'+ ip_address +'/MIMIC_ICU'

# 1. Start with Patients
- DOB
- ID
- Expire Flag will whether they died in ICU 
- Patients contians exactly: r 38,645 adults and 7,875 neonates = 46520 entries

In [7]:
pd.read_sql_query(""" SELECT count(expire_flag) 
                from PATIENTS
                WHERE expire_flag = 1""", cnx)

,count
0,15759


In [8]:
pd.read_sql_query(""" SELECT count(expire_flag)  
                        from PATIENTS
                            WHERE expire_flag =0""", cnx)

,count
0,30761


TWICE AS MANY PEOPLE SURVIVE AS THE PEOPLE WHO DIE, SO LET'S SEE WHO THEY ARE!

In [5]:
pd.read_sql_query(""" SELECT * from PATIENTS limit 3""", cnx)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,612,646,M,2128-01-05,None,None,None,0
1,613,647,M,2106-03-24,None,None,None,0
2,614,648,M,2139-07-13,None,None,None,0


In [9]:
# to find the age, link Patients (dob) on subject_id, and Admissions on subject_id 
pd.read_sql_query(""" SELECT * from ADMISSIONS limit 3""", cnx)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_ioevents_data,has_chartevents_data
0,90,87,190659,2191-02-25 20:30:00,2191-04-25 15:18:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Private,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1
1,91,88,123010,2111-08-29 03:03:00,2111-09-03 14:24:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,,,,BLACK/AFRICAN AMERICAN,2111-08-29 01:44:00,2111-08-29 02:28:00,S/P MOTOR VEHICLE ACCIDENT-STABBING,0,1,1
2,92,89,188646,2185-06-17 05:22:00,2185-06-21 11:15:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1


Keep
+ subject_id
+ gender
+ dob
+ dod
+ expire flag

In [26]:
### ~~ Table 1 

admit = pd.read_sql_query(""" SELECT Patients.subject_id, Patients.gender, Patients.dob, Patients.dod, Patients.expire_flag, Admissions.*
                        FROM Patients LEFT JOIN Admissions
                        ON Patients.subject_id = Admissions.subject_id
                        ORDER BY admission_type DESC""",cnx)
# GROUP BY 1

In [ ]:
admit.to_csv("admit_1.csv", index=None)

In [61]:
admit[:1]

,subject_id,gender,dob,dod,expire_flag,row_id,subject_id,hadm_id,admittime,dischtime,...,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_ioevents_data,has_chartevents_data
0,30,M,1872-10-14 00:00:00,NaT,0,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,...,,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,None,None,UNSTABLE ANGINA\CATH,0,1,1


In [9]:
#admit.info()

# 2. diagnoses-icd 
 ** Find the most common icd9_codes for fatal cases ***
1. try selecting the table
1. select most common icd9 code

In [9]:
## Run Test way down lbelow
test.table_name[test.table_name.str.contains('icd', na=False)]

96        diagnoses_icd
135     d_icd_diagnoses
139    d_icd_procedures
173      procedures_icd
Name: table_name, dtype: object

In [5]:
### look at icd9 codes to get the diagnosis
### Table 2 



,row_id,subject_id,hadm_id,seq_num,icd9_code
0,243,34,115799,8,E8790
1,244,34,144319,1,42789
2,245,34,144319,2,42822
3,246,34,144319,3,4263


In [35]:
# most common code
pd.read_sql_query(''' SELECT count(icd9_code) FROM diagnoses_icd ''',cnx)

,count
0,651047


### Must connect Expire_flag =1 . value_count(icd9_) among ADMISSIONS, not patients

+ Which icd9_code is most common among mortality cases (expire_flag = 1) for Admissions?

+ select  subject_id/hadm_id on Admissions and patients where e_f = 1
+ ** THEN look into the icd9_codes ** 


In [6]:
###### ~~ TABLE 2: 2nd saved table


#### get the most common icd9_codes in JOIN admissions/Patients
# ## Takes a while, but this works: 719641 rows × 4 columns

icd9_expire1 = pd.read_sql_query(''' SELECT admissions.hadm_id, admissions.subject_id, patients.expire_flag, diagnoses_icd.icd9_code
                        From admissions 
                        LEFT JOIN Patients 
                            ON admissions.subject_id = patients.subject_id
                        LEFT JOIN diagnoses_icd 
                            ON diagnoses_icd.subject_id = admissions.subject_id
                        WHERE expire_flag = 1
                    ''', cnx)
icd9_expire1[:5]

,hadm_id,subject_id,expire_flag,icd9_code
0,121205,91,1,20008
1,121205,91,1,2765
2,121205,91,1,5559
3,121205,91,1,99812
4,121205,91,1,51881


In [7]:
icd9_expire1.to_csv("icd9_expire1.csv", index=None)

In [19]:
icd9_expire0 = pd.read_sql_query(''' SELECT admissions.hadm_id, admissions.subject_id, patients.expire_flag, diagnoses_icd.icd9_code
                        From admissions 
                        LEFT JOIN Patients 
                            ON admissions.subject_id = patients.subject_id
                        LEFT JOIN diagnoses_icd 
                            ON diagnoses_icd.subject_id = admissions.subject_id
                        WHERE expire_flag = 0
                    ''', cnx)
icd9_expire0[:5]

,hadm_id,subject_id,expire_flag,icd9_code
0,163353,2,0,V3001
1,163353,2,0,V053
2,163353,2,0,V290
3,185777,4,0,042
4,185777,4,0,1363


In [20]:
icd9_expire0.to_csv("icd9_expire0.csv", index=None)

## Obtain dictionary for icd9 codes

In [11]:
###### ~~ TABLE 4:  saved table

## Procedures
sql = """ SELECT * 
            from d_icd_procedures           
"""

procedures = pd.read_sql(sql, cnx)

In [5]:
#procedures.to_csv("mimic_data/procedures.csv", index=None)

In [16]:
###### ~~ TABLE 5:  saved table

# Diagrnoses AND the definitionbs for the DIagnoses from {d_icd_diagnoses}
sql = """ SELECT * 
            from diagnoses_icd 
            LEFT JOIN d_icd_diagnoses
                ON diagnoses_icd.icd9_code = d_icd_diagnoses.icd9_code
"""

diagnoses = pd.read_sql(sql, cnx)

In [17]:
diagnoses[:3]

,row_id,subject_id,hadm_id,seq_num,icd9_code,row_id,icd9_code,short_title,long_title
0,243,34,115799,8.0,E8790,13429.0,E8790,Abn react-cardiac cath,Cardiac catheterization as the cause of abnorm...
1,244,34,144319,1.0,42789,4471.0,42789,Cardiac dysrhythmias NEC,Other specified cardiac dysrhythmias
2,245,34,144319,2.0,42822,4477.0,42822,Chr systolic hrt failure,Chronic systolic heart failure


In [15]:
diagnoses[:3]

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,243,34,115799,8.0,E8790
1,244,34,144319,1.0,42789
2,245,34,144319,2.0,42822


In [18]:
diagnoses.to_csv("mimic_data/diagnoses.csv", index=None)

# I think Labevents will have the necessary Labs I'm looking for
Number of rows: 27,872,575

SO it might be smarter, to find first either 
 + what patients I'm interested in
 + or what itemid's I'm interested in 
 
** ITEMID --
Identifier for a single measurement type in the database. Each row associated with one ITEMID (e.g. 212) corresponds to an instantiation of the same measurement (e.g. heart rate). **

In [5]:
pd.read_sql_query(""" SELECT *  
                    from LABEVENTS limit 5
                            """, cnx)


,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,None,50820,2101-10-12 17:07:00,7.39,7.39,units,
1,282,3,None,50800,2101-10-12 19:17:00,ART,NaN,,
2,283,3,None,50802,2101-10-12 19:17:00,-1,-1.00,mEq/L,
3,284,3,None,50804,2101-10-12 19:17:00,22,22.00,mEq/L,
4,285,3,None,50808,2101-10-12 19:17:00,0.93,0.93,mmol/L,abnormal


In [24]:
## information on just one subject? 
len(tmp)

1716

In [ ]:
# start = time.time()
# pd.read_sql_query(""" SELECT *  from labevents """, cnx)
# print("total time: ", time.time()-start)

In [16]:
test.table_name[test.table_name.str.contains('lab', na=False)]

21             pg_available_extensions
22     pg_available_extension_versions
25                        pg_seclabels
116                        pg_seclabel
117                      pg_shseclabel
149                         d_labitems
153                          labevents
Name: table_name, dtype: object

In [9]:
start = time.time()
lab = pd.read_sql_query(""" SELECT *  from labevents """, cnx)
print("total time: ", time.time()-start)

('total time: ', 22.716622829437256)


## D_LABITEMS
Table purpose: Definition table for all laboratory measurements.
Number of rows: 755

In [7]:
labs = ['aniongap_min', 'aniongap_max',
       'albumin_min', 'albumin_max', 'bicarbonate_min', 'bicarbonate_max',
       'bilirubin_min', 'bilirubin_max', 'creatinine_min', 'creatinine_max',
        'chloride_min', 'chloride_max', 
    'hematocrit_min', 'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max',
        'lactate_min', 'lactate_max', 'platelet_min', 'platelet_max',
           'potassium_min', 'potassium_max', 'ptt_min', 'ptt_max', 'inr_min',
        'inr_max', 'pt_min', 'pt_max', 'sodium_min', 'sodium_max', 'bun_min',
        'bun_max', 'wbc_min', 'wbc_max','sysbp_max', 'sysbp_mean', 'diasbp_min', 'diasbp_max', 'diasbp_mean',
        'meanbp_min', 'meanbp_max', 'meanbp_mean', 'resprate_min',
        'resprate_max', 'resprate_mean', 'tempc_min', 'tempc_max', 'tempc_mean',
        'spo2_min', 'spo2_max', 'spo2_mean']

In [ ]:
('Anion gap', 'Respiratory Rate (Total)')
#NBP non-invasive BP 

#A BUN test is done to see how well your kidneys are working. 
#If your kidneys are not able to remove urea from the blood normally, 
#your BUN level rises. Heart failure, dehydration, or a diet high in protein can also make your BUN level higher. Liver disease or damage can lower your BUN level.

In [27]:

# Anion gap,
placeholder= '?'
lab_exact = ( 'Respiratory Rate (Total)' ','.join(placeholder * len(l))

In [48]:
#('Anion gap', 'Respiratory Rate (Total)')
## d_items
sql = """ SELECT itemid, label
        from d_items
        where label IN ( 'Anion gap', 'albumin','bicarbonate','Total Bilirubin','Creatinine' ,'chloride','hematocrit',
        'Hemoglobin','CEREBRAL LACTATE','Platelets','Potassium','Sodium','BUN', 'NBP [Systolic]','NBP [Diastolic]', 
        'Temperature C (calc)', 'Respiratory Rate (Total)', 'SpO2')
        
         """
labs = pd.read_sql_query(sql, cnx)

In [18]:
sql = """ SELECT label
        from d_items
        where label='Respiratory Rate (Total)'
        
         """
pd.read_sql_query(sql, cnx)

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,13246,224690,Respiratory Rate (Total),Respiratory Rate (Total),metavision,chartevents,Respiratory,insp/min,Numeric,None


In [52]:
labs


,itemid,label
0,455,NBP [Systolic]
1,1162,BUN
2,646,SpO2
3,677,Temperature C (calc)
4,814,Hemoglobin
5,828,Platelets
6,3066,albumin
7,2638,CEREBRAL LACTATE
8,1525,Creatinine
9,1535,Potassium


In [51]:
labs.itemid.values

array([   455,   1162,    646,    677,    814,    828,   3066,   2638,
         1525,   1535,   1536,   8441,  30006,  41956,  46362, 225170,
       227073, 220228, 224690, 225690, 225624, 220615,  45403])

In [15]:
chart_tables = [u'chartevents', u'chartevents_1', u'chartevents_10',
 u'chartevents_11', u'chartevents_12', u'chartevents_13', u'chartevents_14',
 u'chartevents_2', u'chartevents_3', u'chartevents_4', u'chartevents_5', u'chartevents_6', u'chartevents_7',
 u'chartevents_8', u'chartevents_9']

In [ ]:
# Can I select Chartevents for just one patient?

In [57]:
start = time.time()
sql = """ SELECT subject_id, hadm_id, itemid, mean(valuenum) as val_mean, 
            min(valuenum) as val_min, max(valuesnum) as val_max
        from chartevents_1
        group by hadm_id
        where itemid= 455
         """
pd.read_sql_query(sql, cnx)
#print("total time: ", time.time()-start)

In [58]:
start = time.time()
sql = """ SELECT subject_id, hadm_id, itemid, mean(valuenum) as val_mean, 
            min(valuenum) as val_min, max(valuesnum) as val_max
        from chartevents_1
        group by hadm_id
        where itemid IN (  455,   1162,    646,    677,    814,    828,   3066,   2638,
         1525,   1535,   1536,   8441,  30006,  41956,  46362, 225170,
       227073, 220228, 224690, 225690, 225624, 220615,  45403)
         """
pd.read_sql_query(sql, cnx)
#print("total time: ", time.time()-start)

In [ ]:
start = time.time()
sql = """ SELECT itemid, count(distinct subject_id) as num_patients
        from chartevents_1
        group by itemid
         """
df = pd.read_sql_query(sql, cnx)
df.to_csv('chartevents_1.csv', index=None)
print("total time: ", time.time()-start)

In [14]:
df

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,72456633,7447,128477,227749,198,2136-02-08 10:00:00,2136-02-08 09:51:00,17701,14,14.0,points,None,None,,NotStopd
1,72456937,7447,128477,227749,52,2136-02-08 22:00:00,2136-02-08 22:19:00,18234,102,102.0,mmHg,None,None,,NotStopd
2,72456938,7447,128477,227749,159,2136-02-08 22:00:00,2136-02-08 22:19:00,18234,None,NaN,,None,None,,NotStopd
3,72456939,7447,128477,227749,161,2136-02-08 22:00:00,2136-02-08 22:19:00,18234,None,NaN,,None,None,,NotStopd
4,72456947,7447,128477,227749,52,2136-02-08 23:00:00,2136-02-08 23:14:00,18234,82,82.0,mmHg,None,None,,NotStopd
5,72456948,7447,128477,227749,159,2136-02-08 23:00:00,2136-02-08 23:14:00,18234,None,NaN,,None,None,,NotStopd
6,72456949,7447,128477,227749,161,2136-02-08 23:00:00,2136-02-08 23:14:00,18234,None,NaN,,None,None,,NotStopd
7,72456957,7447,128477,227749,52,2136-02-09 00:00:00,2136-02-09 00:16:00,18234,83,83.0,mmHg,None,None,,NotStopd
8,72456958,7447,128477,227749,77,2136-02-09 00:00:00,2136-02-09 02:21:00,18234,Restless,NaN,,None,None,,NotStopd
9,72456959,7447,128477,227749,127,2136-02-09 00:00:00,2136-02-09 02:21:00,18234,Circulation Adeq,NaN,,None,None,,NotStopd


In [6]:
pd.read_sql_query(""" SELECT *  
                    from D_LABITEMS
                     """, cnx)

,row_id,itemid,label,fluid,category,loinc_code
0,262,51062,"CHLORIDE, STOOL",STOOL,CHEMISTRY,15158-9
1,263,51063,"OSMOLALITY, STOOL",STOOL,CHEMISTRY,2693-0
2,264,51064,"POTASSIUM, STOOL",STOOL,CHEMISTRY,15202-5
3,265,51065,"SODIUM, STOOL",STOOL,CHEMISTRY,15207-4
4,266,51066,24 HR CALCIUM,URINE,CHEMISTRY,6874-2
5,267,51067,24 HR CREATININE,URINE,CHEMISTRY,2162-6
6,268,51068,24 HR PROTEIN,URINE,CHEMISTRY,2889-4
7,269,51069,"ALBUMIN, URINE",URINE,CHEMISTRY,1754-1
8,270,51070,"ALBUMIN/CREATININE, URINE",URINE,CHEMISTRY,14958-3
9,271,51071,"AMPHETAMINE SCREEN, URINE",URINE,CHEMISTRY,3349-8


### rejected queries

In [11]:
sql = """ SELECT *  
        from D_LABITEMS
        WHERE label = 'resprate'"""
pd.read_sql_query(sql, cnx)

,row_id,itemid,label,fluid,category,loinc_code


In [10]:
sql = """ SELECT *  
        from D_LABITEMS
        WHERE label = '24 HR PROTEIN'"""
pd.read_sql_query(sql, cnx)

,row_id,itemid,label,fluid,category,loinc_code
0,268,51068,24 HR PROTEIN,URINE,CHEMISTRY,2889-4


In [5]:
## Code correct, but RUNS OUT OF STORAGE

# ## WIthout Limit 100: how many rows are there?

# sql = """SELECT  patients.expire_flag, diagnoses_icd.subject_id, diagnoses_icd.hadm_id,
#                                 diagnoses_icd.icd9_code
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     WHERE expire_flag=1 """
                    
# pd.read_sql(sql, cnx)

In [4]:
# Code correct, but RUNS OUT OF STORAGE
# sql = """  With new AS (SELECT  patients.expire_flag, diagnoses_icd.subject_id, diagnoses_icd.hadm_id,
#                                 diagnoses_icd.icd9_code
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     WHERE expire_flag=1 )
#  SELECT icd9_code, count(*) as num_icd
#                     FROM new
#                     GROUP BY icd9_code 
#                     order by num_icd desc"""
                    
# pd.read_sql(sql, cnx)

In [3]:
# ## TABLE 3  interrupted takes too long
# pd.read_sql_query(''' With new( SELECT  patients.expire_flag, diagnoses_icd.subject_id, diagnoses_icd.hadm_id,
#                                 diagnoses_icd.icd9_code
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     WHERE expire_flag=1 LIMIT 100)
#                     SELECT icd9_code, count(*)
#                     FROM new
#                     GROUP BY icd9_code''',cnx)

In [2]:
## TABLE 3 Doens't work
# pd.read_sql_query(''' select icd9_code, count(*) from (SELECT  patients.expire_flag, diagnoses_icd.subject_id, diagnoses_icd.hadm_id,
#                                 diagnoses_icd.icd9_code
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     WHERE expire_flag=1)
#                     group by icd9_code
#                     ''',cnx)

In [1]:
# Takes too long
# total number of icd9_codes
# pd.read_sql_query('''LIMIT 10 
#                 SELECT COUNT(DISTINCT icd9_code)
#                     FROM diagnoses_icd
#                     ''',cnx)

In [6]:
## # RUNS OUT OF STORAGE (POSSIBLY INCORRECT)

# pd.read_sql_query('''SELECT COUNT(DISTINCT diagnoses_icd.icd9_code)
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     GROUP BY patients.expire_flag
#                     HAVING patients.expire_flag =1
#                     ''',cnx)

In [7]:
## # RUNS OUT OF STORAGE

## Table 4: like Table 3 but sort according to most common icd9_code
 # (psycopg2.OperationalError) could not write block 1709405 of temporary file: No space left on device

# pd.read_sql_query('''SELECT COUNT(DISTINCT diagnoses_icd.icd9_code)
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     WHERE patients.expire_flag =1
#                     ''',cnx)

In [8]:
## # RUNS OUT OF STORAGE


## Table 4: like Table 3 but sort according to most common icd9_code
 # (psycopg2.OperationalError) could not write block 1709405 of temporary file: No space left on device

# pd.read_sql_query(''' SELECT patients.expire_flag, 
#                             COUNT(DISTINCT diagnoses_icd.icd9_code)
#                     FROM patients 
#                     LEFT JOIN diagnoses_icd 
#                         ON diagnoses_icd.subject_id=diagnoses_icd.subject_id
#                     GROUP BY patients.expire_flag
#                     LIMIT 10''',cnx)

# GROUP BY patients.expire_flag
#                    ORDER BY patients.expire_flag, icd9   DESC

In [ ]:
                       # LEFT JOIN diagnoses_icd.subject_id = patients.subject_id
                                ,diagnoses_icd.icd9_code

In [59]:
#### Don't need 
# pd.read_sql_query(''' SELECT admissions.hadm_id, admissions.subject_id, patients.expire_flag
#                         From admissions 
#                         LEFT JOIN Patients 
#                             ON admissions.subject_id = patients.subject_id
#                         WHERE expire_flag = 1 limit 2
#                     ''', cnx)

,hadm_id,subject_id,expire_flag
0,145834,3,1
1,150750,9,1


In [8]:
# get the subject ID's where expire_flag = 1 and 

# ### Which icd9_code is most common among mortality cases (expire_flag = 1)
# pd.read_sql_query(''' SELECT diagnoses_icd.icd9_code, g, admissions.subject_id
#                     FROM diagnoses_icd LEFT JOIN admissions
#                     ON Diagnoses.subject_id
#                     WHERE expire_flag = 1
#                     ORDER BY count(icd9_code) limit 10''',cnx)

# Table d_icd_diagnose
+ does the d_ stand for something? 
+ diagnoses_icd
        96        diagnoses_icd
        135     d_icd_diagnoses
        139    d_icd_procedures
        173      procedures_icd
        Name: table_name, dtype: obje

In [17]:
#pd.read_sql_query(""" SELECT * from d_icd_diagnoses """, cnx)
pd.read_sql_query(""" SELECT * from diagnoses_icd order by subject_id, hadm_id, seq_num limit 10 """, cnx)

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1,V3001
1,2,2,163353,2,V053
2,3,2,163353,3,V290
3,4,3,145834,1,0389
4,5,3,145834,2,78559
5,6,3,145834,3,5849
6,7,3,145834,4,4275
7,8,3,145834,5,41071
8,9,3,145834,6,4280
9,10,3,145834,7,6826


# Procedures Table to find what they were in for

In [ ]:
pd.read_sql_query(""" SELECT * from d_icd_procedures""", cnx)

In [ ]:
pd.read_sql_query(""" SELECT count(long_title) from d_icd_procedures""", cnx)

In [ ]:
#d_icd_procedures table 
pd.read_sql_query(""" SELECT count( short_title) from d_icd_procedures""", cnx)

In [ ]:
#cnx = create_engine("postgresql://localhost/postgres")

In [ ]:
pd.read_sql_query('''SELECT count(ROW_ID) from PATIENTS''',cnx)

## select all the tables? : test variable

In [5]:
tables= pd.read_sql_query('''SELECT * FROM information_schema.tables''',cnx)

In [6]:
tables.columns

Index([u'table_catalog', u'table_schema', u'table_name', u'table_type',
       u'self_referencing_column_name', u'reference_generation',
       u'user_defined_type_catalog', u'user_defined_type_schema',
       u'user_defined_type_name', u'is_insertable_into', u'is_typed',
       u'commit_action'],
      dtype='object')

In [7]:
tables.head(2)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,MIMIC_ICU,public,w_chartevents_3,BASE TABLE,None,None,None,None,None,YES,NO,None
1,MIMIC_ICU,public,w_chartevents_4,BASE TABLE,None,None,None,None,None,YES,NO,None


In [8]:
len(tables.table_name.unique())

216

In [9]:
test = pd.read_sql_query('''SELECT table_name FROM information_schema.tables''',cnx)

In [12]:
# Series of all the available tables
test[:4]
#print(len(test))

,table_name
0,w_chartevents_3
1,w_chartevents_4
2,pg_type
3,w_chartevents_5


In [13]:
charts = test.table_name[test.table_name.str.contains('chartevents', na=False)]

In [14]:
charts

0       w_chartevents_3
1       w_chartevents_4
3       w_chartevents_5
4       w_chartevents_6
5       w_chartevents_2
6      w_chartevents_13
7      w_chartevents_14
34        chartevents_1
35        chartevents_2
36        chartevents_3
37        chartevents_4
38        chartevents_5
39        chartevents_6
40        chartevents_7
60        chartevents_8
61        chartevents_9
62       chartevents_10
63          chartevents
64       chartevents_11
66       chartevents_12
68       chartevents_13
69       chartevents_14
186    w_chartevents_10
187    w_chartevents_11
188    w_chartevents_12
209     w_chartevents_7
210     w_chartevents_9
211     w_chartevents_8
212     w_chartevents_1
214       w_chartevents
Name: table_name, dtype: object

In [15]:
test
icd = test.table_name[test.table_name.str.contains('icd', na=False)]

In [ ]:
icd

# Chartevents
Kind of a specialized table that tracks the charted events for patients

+ what is the difference b/w w_chartevents, and just chartevents?
+ same for extension _3, _4 etc 


In [ ]:
#pd.read_sql_query('''SELECT * from chartevents limit 2''',cnx)

In [ ]:
#pd.read_sql_query('''SELECT count(DISTINCT subject_id) from chartevents limit10''', cnx)

In [ ]:
# Can you select all the patients/how many different patients in chartevents

pd.read_sql_query('''SELECT count( subject_id) from chartevents''', cnx)

In [ ]:
chart_events = test.table_name[test.table_name.str.contains('chartevents', na=False)]

In [ ]:
pd.read_sql_query('''SELECT * from chartevents_1 limit 2''',cnx)

In [ ]:
#chart_events
pd.read_sql_query('''SELECT subject_id from chartevents_1''',cnx)